# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [52]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import time
from scipy.stats import linregress


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
csv_path = "../WeatherPy/citydf.csv"

# Import the books.csv file as a DataFrame
weather_df = pd.read_csv(csv_path, encoding="utf-8")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barrow,71.2906,-156.7887,10.40,79,1,6.69,US,1619445931
1,waspan,14.7419,-83.9717,80.60,83,20,3.09,NI,1619446125
2,pangnirtung,66.1451,-65.7125,37.40,87,90,3.09,CA,1619446126
3,avarua,-21.2078,-159.7750,69.80,100,54,1.03,CK,1619446127
4,yerbogachen,61.2767,108.0108,19.83,73,91,4.65,RU,1619446128


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [41]:

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barrow,71.2906,-156.7887,10.40,79,1,6.69,US,1619445931
1,waspan,14.7419,-83.9717,80.60,83,20,3.09,NI,1619446125
2,pangnirtung,66.1451,-65.7125,37.40,87,90,3.09,CA,1619446126
3,avarua,-21.2078,-159.7750,69.80,100,54,1.03,CK,1619446127
4,yerbogachen,61.2767,108.0108,19.83,73,91,4.65,RU,1619446128


In [42]:
locations = weather_df[["Lat", "Lng"]]

In [43]:
weight  = weather_df["Humidity"].astype(float)

In [55]:
figW = gmaps.figure()

# Create heat layer
heat_layerW = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
figW.add_layer(heat_layerW)


# Display figure
figW

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#  * A max temperature lower than 90 degrees but higher than 70.

#   * Wind speed less than 20 mph.

#   * less than 30 cloudiness.

In [9]:
Weather_Criteria = weather_df[(weather_df["Max Temp"]>70)&(weather_df["Max Temp"]<90)& (weather_df["Wind Speed"]<20)& (weather_df["Cloudiness"]<10)]
WC= Weather_Criteria.dropna()
WC

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,keti bandar,24.1447,67.4497,83.88,70,0,6.82,PK,1619446128
38,rikitea,-34.4187,19.2345,72.00,61,6,0.89,ZA,1619446141
66,arman,40.4272,71.7189,80.60,19,0,2.06,UZ,1619446153
83,almonte,-22.0920,-70.1979,70.77,58,0,3.09,CL,1619446161
88,krasnoselkup,17.1561,-89.0714,78.80,78,0,3.09,BZ,1619446164
107,margate,-33.0153,27.9116,77.00,61,0,3.60,ZA,1619445908
115,mar del plata,19.1059,37.3321,89.78,53,3,4.20,SD,1619446174
154,samusu,10.5669,72.6420,85.37,68,0,2.39,IN,1619446195
156,biloela,13.5928,-89.8275,77.00,83,0,1.16,SV,1619446150
172,kavaratti,-20.3297,-40.2925,84.20,70,0,3.60,BR,1619446115


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
import requests
import json

In [11]:
hotel_df = pd.read_csv("../WeatherPy/citydf.csv")
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barrow,71.2906,-156.7887,10.40,79,1,6.69,US,1619445931
1,waspan,14.7419,-83.9717,80.60,83,20,3.09,NI,1619446125
2,pangnirtung,66.1451,-65.7125,37.40,87,90,3.09,CA,1619446126
3,avarua,-21.2078,-159.7750,69.80,100,54,1.03,CK,1619446127
4,yerbogachen,61.2767,108.0108,19.83,73,91,4.65,RU,1619446128


In [12]:
hotel_df["Hotel Lat"]= ""
hotel_df["Hotel Lng"]=""
hotel_df["Hotel Name"] = ""
# hotel_df["Hotel Address"] = ""
# hotel_df["Hotel Rating"] = ""
hotel_df.head()




,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Lat,Hotel Lng,Hotel Name
0,barrow,71.2906,-156.7887,10.40,79,1,6.69,US,1619445931,,,
1,waspan,14.7419,-83.9717,80.60,83,20,3.09,NI,1619446125,,,
2,pangnirtung,66.1451,-65.7125,37.40,87,90,3.09,CA,1619446126,,,
3,avarua,-21.2078,-159.7750,69.80,100,54,1.03,CK,1619446127,,,
4,yerbogachen,61.2767,108.0108,19.83,73,91,4.65,RU,1619446128,,,


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
hotel_df.head(100)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Lat,Hotel Lng,Hotel Name
0,barrow,71.2906,-156.7887,10.40,79,1,6.69,US,1619445931,,,
1,waspan,14.7419,-83.9717,80.60,83,20,3.09,NI,1619446125,,,
2,pangnirtung,66.1451,-65.7125,37.40,87,90,3.09,CA,1619446126,,,
3,avarua,-21.2078,-159.7750,69.80,100,54,1.03,CK,1619446127,,,
4,yerbogachen,61.2767,108.0108,19.83,73,91,4.65,RU,1619446128,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
95,arraial do cabo,-19.5500,169.2667,76.37,79,100,5.18,VU,1619446167,,,
96,diamantina,7.7029,27.9953,91.76,38,87,2.85,SS,1619446167,,,
97,san ignacio,18.6853,12.9164,106.72,6,2,4.31,NE,1619445976,,,
98,san cristobal,51.3813,1.3862,48.99,52,100,3.58,GB,1619446168,,,


In [35]:
# params dictionary to update each iteration

params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
   
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    locations = requests.get(base_url, params=params).json()
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = locations["results"][0]["name"]
        hotel_df.loc[index, "Hotel Lat"] = locations["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Hotel Lng"] = locations["results"][0]["geometry"]["location"]["lng"]
        #print(locations)
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

In [36]:
hotel_df.head(100)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Lat,Hotel Lng,Hotel Name
0,barrow,71.2906,-156.7887,10.40,79,1,6.69,US,1619445931,71.297853,-156.771157,Top of the World Hotel
1,waspan,14.7419,-83.9717,80.60,83,20,3.09,NI,1619446125,14.744297,-83.971995,El Piloto Hotel
2,pangnirtung,66.1451,-65.7125,37.40,87,90,3.09,CA,1619446126,66.147679,-65.713338,Auyuittuq Lodge (Inns North)
3,avarua,-21.2078,-159.7750,69.80,100,54,1.03,CK,1619446127,-21.199109,-159.795362,The Islander Hotel
4,yerbogachen,61.2767,108.0108,19.83,73,91,4.65,RU,1619446128,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
95,arraial do cabo,-19.5500,169.2667,76.37,79,100,5.18,VU,1619446167,-19.550735,169.279006,Tanna Ikamir Guest House
96,diamantina,7.7029,27.9953,91.76,38,87,2.85,SS,1619446167,7.713394,27.960601,Amarula Lodge and Restaurant
97,san ignacio,18.6853,12.9164,106.72,6,2,4.31,NE,1619445976,,,
98,san cristobal,51.3813,1.3862,48.99,52,100,3.58,GB,1619446168,51.362328,1.444398,Bay Tree Hotel


In [37]:
hotel_df.to_csv("Hotels.csv")

# Visualize to confirm airport data appears
hotel_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Lat,Hotel Lng,Hotel Name
0,barrow,71.2906,-156.7887,10.40,79,1,6.69,US,1619445931,71.297853,-156.771157,Top of the World Hotel
1,waspan,14.7419,-83.9717,80.60,83,20,3.09,NI,1619446125,14.744297,-83.971995,El Piloto Hotel
2,pangnirtung,66.1451,-65.7125,37.40,87,90,3.09,CA,1619446126,66.147679,-65.713338,Auyuittuq Lodge (Inns North)
3,avarua,-21.2078,-159.7750,69.80,100,54,1.03,CK,1619446127,-21.199109,-159.795362,The Islander Hotel
4,yerbogachen,61.2767,108.0108,19.83,73,91,4.65,RU,1619446128,,,
5,soyo,-6.1349,12.3689,87.80,74,75,3.60,AO,1619446128,-6.163798,12.352401,"Kinwica Resort-Hotel (Soyo, Angola)"
6,keti bandar,24.1447,67.4497,83.88,70,0,6.82,PK,1619446128,,,
7,port elizabeth,-33.9180,25.5701,64.40,77,0,8.75,ZA,1619445948,-33.94592,25.593,Island Way Villa
8,amderma,-33.6500,115.3333,61.00,86,100,0.45,AU,1619446129,-33.659693,115.278613,The Sebel Busselton
9,busselton,65.2820,-126.8329,12.20,67,90,4.12,CA,1619446129,65.277919,-126.813215,Heritage Hotel


In [67]:
figW

locationsH = hotel_df[['Lat','Lng']]
marker_layer = gmaps.marker_layer(locationsH)

# Add marker layer ontop of heat map

figW.add_layer(marker_layer)

# Display figure


# Display figure
figW

Figure(layout=FigureLayout(height='420px'))